In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
# Generate simulation data #

Random.seed!(123);
N = 1200 # sample size
Nobs = 1000;
q = 10; K = 50; 

sqrt_Σ_diag = sqrt.([0.5, 1, 0.4, 2, 0.6, 2.5, 3.0, 0.45, 1.5, 0.5]);

β = [1.0 -1.0 1.0 -0.5 2.0 -1.5 0.5 0.3 -2.0 1.5
     -5.0 2.0 3.0 -2.0 -6.0 4.0 5.0 -3.0 6.0 -4.0
     8.0 6.9 -12.0 0.0 -4.0 7.7 -8.8 3.3 6.6 -5.5];

ϕ = rand(Uniform(3 / sqrt(2), 20 / sqrt(2)), K);
Λ = rand(Uniform(-0.5, 0.5), K, q); # loading matrix
coords = rand(N, 2);                                          # random location over unit square
ordx = 1:N;
coords_ord = coords[ordx, :];
S_ind = [sample(1:N, Nobs, replace = false, ordered = true) for i in 1:q] 
                                                              # index of observed response for all q responses
S = sort(union([S_ind[i] for i in 1:q]...));                  # location with at least one observation
M_ind = [setdiff(S, S_ind[i]) for i in 1:q];                  # in S not in S1
obs_ind = vcat([S_ind[i] .+ (i - 1) * N for i in 1:q]...)     # index of the observed location for all response among N locations
perm_ind = sortperm(vcat([S_ind[i] for i in 1:q]...))

index_S = zeros(N);                                           # build index indicating which response are observed
index_S_M = fill(0.0, N, q);
for i in 1:q
    index_S_M[S_ind[i], i] .= 1;
    index_S = index_S + 2^(i - 1) * index_S_M[:, i]
end
M_Sind = [findall(x -> x == 0.0, index_S_M[S, i]) for i in 1:q];

In [4]:
# Some data preparations #

m = 10; n = length(S); 
# number of nearest neighbor                       
NN = BuildNN(coords_ord[S, :], m);                            # build nearest neighbor 
nnIndx_col = vcat(NN.nnIndx, 1:n);                            # the index of columns
nnIndx_row = zeros(Int64, 0);                                               
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1));
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n);  # the index of rows


In [5]:
@save "sim3_pars.jld"

In [3]:
@load "sim3_pars.jld"

45-element Array{Symbol,1}:
 Symbol("##1#2")   
 Symbol("##11#12") 
 Symbol("##13#14") 
 Symbol("##15#16") 
 Symbol("##17#18") 
 Symbol("##19#21") 
 Symbol("##20#22") 
 Symbol("##3#5")   
 Symbol("##4#6")   
 Symbol("##7#8")   
 Symbol("##9#10")  
 Symbol("#BuildNN")
 Symbol("#colnorm")
 ⋮                 
 :m                
 :n                
 :nnIndx_col       
 :nnIndx_row       
 :obs_ind          
 :ordx             
 :perm_ind         
 :q                
 :sqrt_Σ_diag      
 :Λ                
 :β                
 :ϕ                

In [21]:
round.(ϕ[1:10], digits = 2)

10-element Array{Float64,1}:
 11.36
 13.43
 10.22
  6.87
  5.89
 10.09
  9.17
  2.75
  5.35
  3.43

In [22]:
round.(ϕ[11:20], digits = 2)

10-element Array{Float64,1}:
  4.09
  7.81
 12.52
  9.54
  5.56
  7.7 
  5.44
  7.49
  9.12
  5.2 

In [23]:
round.(ϕ[21:30], digits = 2)

10-element Array{Float64,1}:
 10.61
  5.63
  5.5 
 11.65
  4.64
 13.16
  9.51
 11.77
  8.8 
 13.43

In [24]:
round.(ϕ[31:40], digits = 2)

10-element Array{Float64,1}:
  7.89
 11.62
  6.4 
 12.95
  8.48
  2.5 
 12.95
 13.42
  9.59
  6.31

In [25]:
round.(ϕ[41:50], digits = 2)

10-element Array{Float64,1}:
  8.98
  4.57
  6.63
 11.25
  4.43
  4.94
  3.3 
  9.66
 13.5 
  8.7 

In [16]:
round.(Λ[1:10, 1:10], digits = 2)

10×10 Array{Float64,2}:
 -0.38  -0.33   0.23  -0.38  -0.13   0.31   0.28   0.0    0.42   0.18
 -0.39  -0.13  -0.13  -0.31  -0.15   0.42   0.13  -0.07   0.21   0.1 
  0.01   0.48   0.32   0.13  -0.46   0.27   0.09   0.12   0.25   0.38
  0.13   0.48  -0.47  -0.48  -0.34  -0.09  -0.28  -0.21  -0.19   0.44
 -0.47   0.46   0.24  -0.45   0.44  -0.29  -0.36  -0.46   0.34  -0.44
  0.21   0.12  -0.46   0.29   0.36   0.17   0.03   0.2   -0.12  -0.23
 -0.2    0.48   0.18  -0.1    0.13  -0.13  -0.41  -0.04  -0.07  -0.22
 -0.19   0.28   0.47  -0.42   0.17  -0.18  -0.03  -0.13  -0.04   0.3 
 -0.04   0.27  -0.23  -0.07  -0.09  -0.39  -0.48  -0.27   0.19   0.21
 -0.03  -0.18  -0.08  -0.12   0.35   0.3   -0.33   0.34   0.38   0.31

In [17]:
round.(Λ[11:20, 1:10], digits = 2)

10×10 Array{Float64,2}:
 -0.15  -0.43   0.27   0.18   0.38  -0.4   -0.27  -0.3   -0.38  -0.09
  0.28  -0.24  -0.32  -0.08  -0.01  -0.31   0.2    0.31   0.11  -0.38
 -0.38  -0.42  -0.16  -0.37  -0.22   0.09  -0.08  -0.07  -0.33   0.01
 -0.42  -0.22   0.44   0.09   0.25   0.12   0.1   -0.33  -0.41   0.42
  0.42   0.48  -0.06   0.07   0.43   0.12  -0.15   0.29   0.1   -0.32
 -0.15  -0.03  -0.42   0.01   0.05   0.33  -0.46   0.12   0.22  -0.44
  0.28  -0.08  -0.41   0.13   0.03   0.22   0.08   0.32   0.02  -0.41
  0.35  -0.39   0.37  -0.47  -0.08  -0.01   0.09   0.06  -0.21   0.38
 -0.38   0.2   -0.21   0.21  -0.11   0.27   0.2    0.17   0.31  -0.12
  0.36  -0.09  -0.16  -0.06   0.43  -0.04  -0.07   0.4   -0.39  -0.06

In [18]:
round.(Λ[21:30, 1:10], digits = 2)

10×10 Array{Float64,2}:
  0.08   0.15   0.11   0.37   0.25   0.28   0.13  -0.18   0.35   0.17
 -0.32  -0.31   0.24  -0.29  -0.38  -0.1   -0.19   0.18  -0.37  -0.34
  0.45   0.19   0.34  -0.36   0.43   0.44  -0.13  -0.26  -0.46  -0.08
  0.38  -0.48  -0.22  -0.14   0.5    0.08   0.02  -0.07   0.07  -0.3 
 -0.49  -0.48   0.34   0.1   -0.01   0.2    0.33   0.37   0.1    0.21
  0.1    0.11  -0.33  -0.16   0.06   0.25  -0.37  -0.1   -0.16  -0.13
  0.45   0.02  -0.21   0.16   0.37  -0.2   -0.44  -0.37   0.46   0.25
  0.34   0.31   0.06  -0.25   0.37   0.12   0.27  -0.35   0.09  -0.28
 -0.2   -0.12  -0.41   0.23  -0.23  -0.07  -0.34   0.37  -0.43   0.18
  0.36   0.14   0.47   0.3    0.36  -0.09   0.1   -0.01   0.11   0.43

In [19]:
round.(Λ[31:40, 1:10], digits = 2)

10×10 Array{Float64,2}:
  0.42   0.17  -0.24   0.05  -0.0   -0.41  -0.03  -0.0   -0.22   0.2 
  0.26  -0.22   0.33  -0.06  -0.06  -0.36  -0.31   0.14  -0.14  -0.1 
  0.09   0.43   0.04  -0.35   0.42   0.19   0.33  -0.12   0.4   -0.32
 -0.13   0.36   0.02   0.02   0.34   0.06  -0.32  -0.47   0.02   0.34
  0.27  -0.35  -0.12   0.5    0.33   0.33  -0.27   0.39   0.45   0.27
  0.38   0.11   0.05   0.38  -0.34  -0.19  -0.12   0.39   0.2    0.31
  0.16   0.31   0.02  -0.43   0.13   0.33  -0.34  -0.1    0.41  -0.46
  0.32  -0.2   -0.18  -0.05   0.2   -0.17  -0.06   0.49  -0.06   0.3 
  0.44  -0.05   0.06  -0.22  -0.16  -0.43   0.04  -0.23  -0.22   0.11
 -0.23  -0.34   0.45  -0.47   0.03  -0.09  -0.47   0.28   0.27  -0.4 

In [20]:
round.(Λ[41:50, 1:10], digits = 2)

10×10 Array{Float64,2}:
  0.28   0.27  -0.45  -0.15   0.05   0.31   0.16   0.49   0.12  -0.43
  0.14  -0.16   0.21  -0.3    0.36  -0.29   0.17   0.16  -0.35  -0.05
 -0.2   -0.31   0.11  -0.46   0.41   0.09  -0.24  -0.21   0.4   -0.05
  0.44   0.44   0.41  -0.22   0.36  -0.45  -0.19   0.46   0.49  -0.28
 -0.34  -0.5   -0.33  -0.37   0.33  -0.31  -0.37   0.05  -0.38  -0.14
  0.33   0.46  -0.35  -0.42  -0.01  -0.48  -0.33  -0.23  -0.07   0.09
  0.21  -0.49  -0.31  -0.04   0.23   0.43   0.22   0.23  -0.25  -0.45
 -0.08   0.35   0.01   0.25  -0.07  -0.29  -0.05   0.19  -0.07  -0.14
 -0.4   -0.38   0.07   0.23   0.43  -0.05   0.08   0.03   0.09   0.02
 -0.13  -0.08  -0.18  -0.02  -0.38  -0.07   0.41   0.18  -0.31   0.35